In [1]:
import random
import pandas as pd
import collections
from fractions import Fraction
import itertools


In [2]:
card_deck = []

i = 2
j = 1
while i < 15 and j < 5:
    card = []
    card.append(i)
    
    if j == 1:
        card.append('Heart')
    elif j == 2:
        card.append('Diamond')
    elif j == 3:
        card.append('Spade')
    elif j == 4:
        card.append('Club')
    
    if i == 2:
        card.append('Y')
    else:
        card.append('N')
        
    if i == 14:
        j = j + 1
        i = 1
    
    card_deck.append(card)
    i = i + 1
    

In [3]:
i = 1
j = 52
card_hand = []

while i < 6:
    x = random.randrange(0, j)    
    dealt_card = card_deck.pop(x)
    card_hand.append(dealt_card)
    i = i + 1
    j = j - 1
    
first_dealt_hand = card_hand
first_dealt_hand

[[5, 'Diamond', 'N'],
 [9, 'Heart', 'N'],
 [12, 'Heart', 'N'],
 [7, 'Club', 'N'],
 [2, 'Heart', 'Y']]

In [4]:
hands_df = pd.read_csv('../Extra_Storage/UPDATED_ALL_HAND_OPTIONS.csv')  
hands_df = hands_df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

discard_list = [0,1,2,3,4]
all_discard_options = []
for L in range(len(discard_list) + 1):
    for subset in itertools.combinations(discard_list, L):
        discard_option = list(subset)
        all_discard_options.append(discard_option)
        
all_discard_options.pop(0)


string_first_hand = []
for card in first_dealt_hand:
    string_first_hand.append(str(card))

string_first_hand


# potential_discarded_card
all_discard_options

complete_discard_list = []

for item in all_discard_options:
    discard_string = ''

    for mini_item in item:
        temp_string = str(mini_item)
        discard_string = discard_string + temp_string
        
    complete_discard_list.append(discard_string)
    
complete_discard_list

complete_discard_odds_df_list = []

for discard_option in complete_discard_list:
    test_discarded_card_position_list = [*discard_option]

    saved_hand = []
    for card in first_dealt_hand:
        saved_hand.append(card)

    temp_hand = saved_hand

    
    k = 0
    discarded_cards = []
    for item in test_discarded_card_position_list:
        num_item = int(item)
        card_position = num_item - k
        discard = temp_hand.pop(card_position)
        discarded_cards.append(discard)
        k = k + 1

    string_hand = []    
    for item in temp_hand:
        new_item = str(item)
        string_hand.append(new_item)

        
    if len(string_hand) == 1:
        test_odds_df = hands_df[hands_df.isin([string_hand[0]]).any(axis=1)]
    elif len(string_hand) == 2:
        test_odds_df = hands_df[hands_df.isin([string_hand[0]]).any(axis=1) & hands_df.isin([string_hand[1]]).any(axis=1)]
    elif len(string_hand) == 3:
        test_odds_df = hands_df[hands_df.isin([string_hand[0]]).any(axis=1) & hands_df.isin([string_hand[1]]).any(axis=1) & hands_df.isin([string_hand[2]]).any(axis=1)]
    elif len(string_hand) == 4:
        test_odds_df = hands_df[hands_df.isin([string_hand[0]]).any(axis=1) & hands_df.isin([string_hand[1]]).any(axis=1) & hands_df.isin([string_hand[2]]).any(axis=1) & hands_df.isin([string_hand[3]]).any(axis=1)]
    elif len(string_hand) == 0:
        test_odds_df = hands_df.loc[~hands_df.isin(string_first_hand).any(axis=1)]


    test_odds_summary_df = pd.DataFrame(test_odds_df['Result'].value_counts())

    test = test_odds_summary_df['Result'].sum()
    
    temp_odds_list = test_odds_summary_df['Result'].tolist()

    new_list = []
    for item in temp_odds_list:
        new_list.append("{:.5f}".format((item / test) * 100))

    test_odds_summary_df['Percentage'] = new_list

    win_total_df = test_odds_summary_df
    
    if "['Loss']" in win_total_df.index:
        win_total_df = win_total_df.drop(index=("['Loss']"))

    win_total = win_total_df['Result'].sum()

    test_odds_summary_df.loc['Win', :] = [win_total,("{:.5f}".format((win_total / test) * 100))]

    
    test_odds_summary_df['Discard Positions'] = discard_option

    discarded_cards_string = str(discarded_cards)
    test_odds_summary_df['Discard Cards'] = discarded_cards_string

    complete_discard_odds_df_list.append(test_odds_summary_df)

complete_discard_odds_df = pd.concat(complete_discard_odds_df_list)

# complete_discard_odds_df
final_complete_discard_odds_df = complete_discard_odds_df.drop(index=("['Loss']"))
final_complete_discard_odds_df['Percentage'] = final_complete_discard_odds_df['Percentage'].astype(float)
final_complete_discard_odds_df = final_complete_discard_odds_df.sort_values(by=['Percentage'], ascending = False)
print(first_dealt_hand)
final_complete_discard_odds_df.head(15)

[[5, 'Diamond', 'N'], [9, 'Heart', 'N'], [12, 'Heart', 'N'], [7, 'Club', 'N'], [2, 'Heart', 'Y']]


/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_7614/3677241383.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_complete_discard_odds_df['Percentage'] = final_complete_discard_odds_df['Percentage'].astype(float)


,Result,Percentage,Discard Positions,Discard Cards
Win,133476.0,53.41176,0123,"[[5, 'Diamond', 'N'], [9, 'Heart', 'N'], [12, ..."
Win,10213.0,52.10714,012,"[[5, 'Diamond', 'N'], [9, 'Heart', 'N'], [12, ..."
Win,10213.0,52.10714,023,"[[5, 'Diamond', 'N'], [12, 'Heart', 'N'], [7, ..."
Win,9835.0,50.17857,123,"[[9, 'Heart', 'N'], [12, 'Heart', 'N'], [7, 'C..."
Win,9835.0,50.17857,013,"[[5, 'Diamond', 'N'], [9, 'Heart', 'N'], [7, '..."
Win,576.0,48.97959,03,"[[5, 'Diamond', 'N'], [7, 'Club', 'N']]"
Win,568.0,48.29932,02,"[[5, 'Diamond', 'N'], [12, 'Heart', 'N']]"
Win,568.0,48.29932,12,"[[9, 'Heart', 'N'], [12, 'Heart', 'N']]"
Win,504.0,42.85714,23,"[[12, 'Heart', 'N'], [7, 'Club', 'N']]"
Win,20.0,41.66667,2,"[[12, 'Heart', 'N']]"


In [5]:
card_position_counter = 0
for card in first_dealt_hand:
    print(f'Card Position: {card_position_counter} {card}')
    card_position_counter = card_position_counter + 1

discarded_card_input = input("Select Cards to Discard by Position Number:")

discarded_card_position_list = [*discarded_card_input]
discarded_card_position_list = [i for i in discarded_card_position_list if i.isnumeric()]

k = 0
discarded_cards = []
for item in discarded_card_position_list:
    num_item = int(item)
    card_position = num_item - k
    discard = card_hand.pop(card_position)
    discarded_cards.append(discard)
    k = k + 1


i = 6 - len(discarded_card_position_list)
j = 47
final_card_hand = card_hand


while i < 6:
    x = random.randrange(0, j)    
    dealt_card = card_deck.pop(x)
    final_card_hand.append(dealt_card)
    i = i + 1
    j = j - 1
    
print(f'Discarded Cards: {discarded_cards}')
print(final_card_hand)

# Winning hands - Odds
winning_hand_indicator = []

# Get card numbers and sort them to make it easier to identify winning patters
wildcard_position_indicator = 0
wildcard_positions = []

for third in final_card_hand:
    
    if third[2] == 'Y':
        wildcard_positions.append(wildcard_position_indicator)
        
    wildcard_position_indicator = wildcard_position_indicator + 1

wildcard_counter = len(wildcard_positions)


# Get card numbers and sort them to make it easier to identify winning patters
final_hand_numbers = []
final_hand_numbers_no_wildcards = []

for first in final_card_hand:
    final_hand_numbers.append(first[0])
    if first[0] == 2:
        continue
    else:
        final_hand_numbers_no_wildcards.append(first[0])
        
final_hand_numbers.sort()
final_hand_numbers_no_wildcards.sort()


# Get card suits and sort them to make it easier to identify winning patters
final_hand_suits = []
final_hand_suits_no_wildcards = []

for second in final_card_hand:
    final_hand_suits.append(second[1])

    if second[0] == 2:
        continue
    else:
        final_hand_suits_no_wildcards.append(second[1])

    
final_hand_suits.sort()

# Four Dueces - 200

if wildcard_counter == 4:
    winning_hand_indicator.append('Four Deuces')
    

# Five of a Kind - 10

card_number_count = collections.Counter(final_hand_numbers_no_wildcards)
if card_number_count[card_number_count.most_common(1)[0][0]] + len(wildcard_positions) == 5:
    winning_hand_indicator.append('Five of a Kind')


# Four of a Kind - 4

card_number_count = collections.Counter(final_hand_numbers_no_wildcards)
if card_number_count[card_number_count.most_common(1)[0][0]] + len(wildcard_positions) == 4:
    winning_hand_indicator.append('Four of a Kind')


# Full House - 3

no_wildcard_numbers_set = set(final_hand_numbers_no_wildcards)

card_number_count = collections.Counter(final_hand_numbers_no_wildcards)
if card_number_count[card_number_count.most_common(1)[0][0]] + len(wildcard_positions) == 3 and len(no_wildcard_numbers_set) == 2:
    winning_hand_indicator.append('Full House')


# Flush - 2
suits_set = set(final_hand_suits)
no_wildcard_suits_set = set(final_hand_suits_no_wildcards)

if len(suits_set) == 1:
    winning_hand_indicator.append('Flush')
    
elif len(wildcard_positions) > 0:
    unique_suits_frequency = []
    for item in suits_set:
        unique_suits_frequency.append(final_hand_suits.count(item))
        if final_hand_suits.count(item) == len(final_hand_suits_no_wildcards) and len(no_wildcard_suits_set) == 1:
            winning_hand_indicator.append('Flush')


# Straight - 2
i = 0
j = 1
temp_wildcard_counter = wildcard_counter
for i in range(len(final_hand_numbers_no_wildcards)):
    if wildcard_counter == 4:
        break
    if final_hand_numbers_no_wildcards[j] - final_hand_numbers_no_wildcards[i] == 1:
        if j == (len(final_hand_numbers_no_wildcards) - 1):
            winning_hand_indicator.append('Straight')
            break
        else:
            i = i + 1
            j = j + 1
            continue
    elif final_hand_numbers_no_wildcards[j] - final_hand_numbers_no_wildcards[i] == 2 and temp_wildcard_counter == 1:
        if j == (len(final_hand_numbers_no_wildcards) - 1):
            winning_hand_indicator.append('Straight')
            break
        else:
            i = i + 1
            j = j + 1
            temp_wildcard_counter = temp_wildcard_counter - 1
            continue
    elif final_hand_numbers_no_wildcards[j] - final_hand_numbers_no_wildcards[i] == 3 and temp_wildcard_counter == 2:
        if j == (len(final_hand_numbers_no_wildcards) - 1):
            winning_hand_indicator.append('Straight')
            break
        else:
            i = i + 1
            j = j + 1
            temp_wildcard_counter = temp_wildcard_counter - 2
            continue
    else:
        i = i + 1
        j = j + 1
        break

        
# Three of a Kind - 1

card_number_count = collections.Counter(final_hand_numbers_no_wildcards)
if card_number_count[card_number_count.most_common(1)[0][0]] + len(wildcard_positions) == 3:
    winning_hand_indicator.append('Three of a Kind')


# Straight Flush - 8

if 'Flush' in winning_hand_indicator and 'Straight' in winning_hand_indicator:
    winning_hand_indicator.remove('Flush')
    winning_hand_indicator.remove('Straight')
    winning_hand_indicator.append('Straight Flush')

    
# Royal Flush with No Duece - 300

if 'Straight Flush' in winning_hand_indicator:
    if 14 in final_hand_numbers_no_wildcards and 10 in final_hand_numbers_no_wildcards and wildcard_counter == 0:
        winning_hand_indicator.remove('Straight Flush')
        winning_hand_indicator.append('Royal Flush')
        

# Royal Flush with Duece - 20

if 'Straight Flush' in winning_hand_indicator:
    royal_flush_numbers = [10,11,12,13,14]
    royal_flush_counter = 0
    for number in final_hand_numbers_no_wildcards:
        if number in royal_flush_numbers:
            royal_flush_counter = royal_flush_counter + 1
            continue
        else:
            break
    if royal_flush_counter == len(final_hand_numbers_no_wildcards):
        winning_hand_indicator.remove('Straight Flush')
        winning_hand_indicator.append('Royal Flush with Duece')

        

# Final Decision
if len(winning_hand_indicator) == 0:
    winning_hand_indicator.append('Loss')

print(winning_hand_indicator)


Card Position: 0 [5, 'Diamond', 'N']
Card Position: 1 [9, 'Heart', 'N']
Card Position: 2 [12, 'Heart', 'N']
Card Position: 3 [7, 'Club', 'N']
Card Position: 4 [2, 'Heart', 'Y']
Select Cards to Discard by Position Number:0123
Discarded Cards: [[5, 'Diamond', 'N'], [9, 'Heart', 'N'], [12, 'Heart', 'N'], [7, 'Club', 'N']]
[[2, 'Heart', 'Y'], [5, 'Club', 'N'], [14, 'Heart', 'N'], [2, 'Spade', 'Y'], [11, 'Spade', 'N']]
['Three of a Kind']
